## PyTorchの画像分類モデルを変換してOpenVINOで使う方法を学ぶ  

このチュートリアルではPyTorchの画像分類モデルをOpenVINO IRモデルに変換する方法について学びます。  

途中でPyTorchで推論をした場合と、OpenVINOで推論をした場合の簡易パフォーマンス比較も行います。OpenVINOにより推論が高速化されることを確認してください。

また、後半では推論プログラムの効率を向上させるカギとなる、非同期推論APIのサンプルコードを見ながら、基本的な使い方を学びます。

## 必要なライブラリのインポート

In [ ]:
import cv2
import numpy as np

import torch
import torchvision

import openvino as ov

from PIL import Image
from IPython.display import display

## PyTorchの学習済みモデルの読み込み (ダウンロード)  

学習済みresnet50モデルをtorch hubからダウンロードします。  
`.eval()` APIを呼び出し、モデルを推論モードにしておきます。

In [ ]:
pt_model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT, progress=True)
pt_model.eval()

## PyTorchモデルをOpenVINO IRモデルに変換し、ファイルとして保存する  

`convert_model()` APIを使用することでモデルを変換することができます。

In [ ]:
ov_model = ov.convert_model(pt_model)
ov.save_model(ov_model, 'resnet50.xml', compress_to_fp16=True)

## 保存したOpenVINO IRモデルファイルを読み込み、デバイス向けにコンパイルする  

読み込んだモデルは入力画像サイズ(H, W)やバッチサイズ(N)が不定(?)のダイナミックシェイプモデルになっています。`reshape()` APIでシェイプを指定し、スタティックシェイプモデルにしています。この処理は行わなくてもモデルの実行は可能です (ダイナミックシェイプのまま実行可能)。ですが、スタティックシェイプにすることでほんの少しパフォーマンスが向上し、メモリ使用量も削減されます。

In [ ]:
ov_model = ov.Core().read_model('resnet50.xml')
print(ov_model.inputs[0])
ov_model.reshape((1,3,224,224))                   # 読み込んだモデルのシェイプは[?,3,?,?]のダイナミックシェイプモデルなので、(1,3,224,224)にリシェイプしてスタティックシェイプモデルにする (オプショナル)
print(ov_model.inputs[0])
compiled_model = ov.compile_model(ov_model, device_name='GPU.0', config={'CACHE_DIR':'./cache'})
compiled_model

## 入力画像の読み込みとプリプロセス

In [ ]:
img = cv2.imread('beaver.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224,224))
display(Image.fromarray(img))
img = img.astype(np.float32) / 255.0 - 0.5
img = np.transpose(img, (2,0,1))
img = np.expand_dims(img, axis=0)
img.shape

## OpenVINOで推論実行

In [ ]:
result = compiled_model.infer_new_request(img)
result = result[0].ravel()
print(result.shape, result[:40])

## 推論結果の表示  

正しい推論結果が出ていることを確認してください。

In [ ]:
with open('synset_words.txt') as f:
    imagenet_labels = [ line.rstrip().split(maxsplit=1)[1] for line in f.readlines() ]

indices = np.argsort(result)[::-1]

for index in indices[:5]:
    print(index, imagenet_labels[index], result[index])

## PyTorchでの推論時間を計測  

同じ入力データを使用してPyTorch (CPU)で推論を実行し時間計測を行います。  
`%%timeit`はJupyter notebookの機能で、続くコマンドを複数回実行し、パフォーマンスデータを表示してくれます。

In [ ]:
# PyTorch
pt_tensor = torch.Tensor(img)

%timeit pt_model(pt_tensor)

## OpenVINOでの推論時間を計測  

同様に、OpenVINOで推論を行った時の推論時間を計測します。PyTorchと実行時間を比較してみてください。

In [ ]:
# OpenVINO - Synchronous inference

%timeit compiled_model(img)

## OpenVINO非同期推論API その１  

ここではコールバック関数を使用せず、非同期推論を行っています。  
`AsyncInferQueue()`で推論要求キューを作成すると、自動でキューの管理を行ってくれるので便利です。これを使わずに`create_infer_request()` APIで推論要求バッファを作成した場合は要求バッファの空きや使用状況管理をユーザープログラム側で行わなくてはならなくなります。  
最後に`async_queue.wait_all()`ですべての要求が処理されたことを確認しています。  

このケースではプリプロセスやポストプロセスもなく、推論処理も軽いため同期推論と非同期推論のパフォーマンス差はほとんどありません。推論プログラムが複雑になり、プリ・ポストプロセスが重くなるにつれ、非同期推論の重要性は増してきますので覚えておいてください。

In [ ]:
%%time
# OpenVINO - Asynchronous inference

async_queue = ov.AsyncInferQueue(compiled_model, jobs=4)
niter = 100

for _ in range(niter):
    #while async_queue.is_ready() == False: pass
    async_queue.start_async(img)

async_queue.wait_all()

## OpenVINO非同期推論API その２  

この例では、推論要求バッファ(`infer_request`)を２つ作成し、交互に推論要求を投入するようにしています。  
ほとんどの推論プログラムではこのような使用方法で十分プリ・ポストプロセス処理を隠ぺいすることが可能です。

In [ ]:
%%time
async_queue = [ compiled_model.create_infer_request() for _ in range(2) ]
curr_id = 0
niter = 100

for _ in range(niter):
    next_id = 1 - curr_id
    async_queue[next_id].wait()
    async_queue[next_id].start_async(img)

for i in range(2):
    async_queue[i].wait()

## ## OpenVINO非同期推論API その３  

この例では推論結果をコールバック関数で受け取るようになっています。これにより推論結果のポーリング的処理が不要となり推論ループは簡潔になりますが、同時に完全な非同期処理が必要となり、全体のシーケンス管理がやや複雑になります。

In [ ]:
%%time
async_queue = [ compiled_model.create_infer_request() for _ in range(2) ]
curr_id = 0
niter = 100

def infer_callback(user_data):
    res, infer_id, postprocess = user_data
    if postprocess:
        # Post processing
        res = res.output_tensors[0].data
        class_id = np.argmax(res)
        print(infer_id, class_id)

for i in range(niter):
    next_id = 1 - curr_id
    async_queue[next_id].wait()
    async_queue[next_id].set_callback(callback=infer_callback, userdata=[async_queue[next_id], i, False])
    async_queue[next_id].start_async(img)

for i in range(2):
    async_queue[i].wait()

## まとめ

ここでは簡単にPyTorchのモデルを`convert_model()` APIで変換し、OpenVINOで利用できることを学びました。  
また、OpenVINOとPyTorchでの推論パフォーマンス比較も行いました。多くの場合、OpenVINOを利用することによって、インテルハードウエアの持つ機能を最大限活用することが可能となり、推論パフォーマンスが向上します。これにより、従来は外付けGPUなどが必要と思われていた処理がPCのCPUでも十分処理できるようになり、応用範囲が広がり、BOMコスト低減に貢献します。

最後に非同期推論APIの利用例も見てきました。プログラムが複雑になり、モデルのプリ・ポスト処理が重くなるほど非同期推論の重要性は増してきます。  
CPU以外の推論デバイスに推論処理をオフロードしたときも非同期推論は重要になります。せっかく推論処理をオフロードしても、推論処理が終わるまでCPUが待ちぼうけているのでは意味がありません。